In [1]:
import sys
sys.path.append("..")

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime
# Import the API key.
from config import weather_api_key
# Import the time module
import time
from scipy.stats import linregress
import gmaps
import random

In [2]:
# make 2000 random latitudes and longitudes

lats = np.random.uniform(low= -90, high= 90, size= 2000)
lngs = np.random.uniform(low= -180, high= 180, size= 2000)

In [3]:
# zip them bitches together in a list
coords = list(zip(lats, lngs))

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coords:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


735

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key



In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kapaa
Processing Record 2 of Set 1 | escanaba
Processing Record 3 of Set 1 | arraial do cabo
Processing Record 4 of Set 1 | xunchang
Processing Record 5 of Set 1 | torbay
Processing Record 6 of Set 1 | tasiilaq
Processing Record 7 of Set 1 | tiksi
Processing Record 8 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 9 of Set 1 | port elizabeth
Processing Record 10 of Set 1 | carnarvon
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | atuona
Processing Record 14 of Set 1 | constitucion
Processing Record 15 of Set 1 | lompoc
Processing Record 16 of Set 1 | diego de almagro
Processing Record 17 of Set 1 | provideniya
Processing Record 18 of Set 1 | hithadhoo
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | kieta
Processing Record 21 of Set 1 | bredasdorp
Processing Record 22 of Set 1 | ati
Process

In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Hilo,19.73,-155.09,87.80,55,40,14.99,US,scattered clouds
1,Arraial Do Cabo,-22.97,-42.03,70.12,73,70,9.08,BR,broken clouds
2,Hovd,48.01,91.64,32.04,79,0,2.71,MN,clear sky
3,Salalah,17.02,54.09,73.40,56,0,3.36,OM,clear sky
4,Barrow,71.29,-156.79,17.60,72,1,26.40,US,clear sky
5,Mount Isa,-20.73,139.50,87.80,16,0,11.41,AU,clear sky
6,Rikitea,-23.12,-134.97,73.87,81,99,19.75,PF,overcast clouds
7,Punta Arenas,-53.15,-70.92,41.00,86,75,10.29,CL,broken clouds
8,Half Moon Bay,37.46,-122.43,93.20,17,1,13.87,US,clear sky
9,Ancud,-41.87,-73.82,59.00,58,54,10.29,CL,broken clouds


In [15]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df=city_data_df[new_column_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hilo,US,19.73,-155.09,87.80,55,40,14.99,scattered clouds
1,Arraial Do Cabo,BR,-22.97,-42.03,70.12,73,70,9.08,broken clouds
2,Hovd,MN,48.01,91.64,32.04,79,0,2.71,clear sky
3,Salalah,OM,17.02,54.09,73.40,56,0,3.36,clear sky
4,Barrow,US,71.29,-156.79,17.60,72,1,26.40,clear sky
...,...,...,...,...,...,...,...,...,...
663,Ciudad Bolivar,VE,8.12,-63.55,84.22,60,57,3.78,broken clouds
664,Nova Olinda Do Norte,BR,-3.89,-59.09,81.46,63,30,1.74,scattered clouds
665,Waterloo,CA,43.47,-80.52,42.80,60,1,6.93,clear sky
666,Katherine,AU,-14.47,132.27,86.00,55,45,8.05,scattered clouds


In [17]:
# Create the output file (CSV).
output_data_file = "WheatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")